In [17]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [18]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [50]:
#df = pd.DataFrame(list(analyseCol.find({'soort': 'Stelling'})))
df = pd.DataFrame(list(analyseColClean.find({'soort': 'Put'})))
df.drop(['ID', 'index', 'level_0'], inplace=True, errors='ignore')
df.reset_index(inplace=True)
df['ID'] = df['index']
df

,index,_id,brondata,projectcd,putnr,soort,key,key_project,ID
0,0,6201144fbba1b3fdae51bef9,"{'_id': 6201144fbba1b3fdae51bef9, 'PUT': 1, 'P...",DC154,1,Put,PDC154P1,PDC154,0
1,1,6201144fbba1b3fdae51bef8,"{'_id': 6201144fbba1b3fdae51bef8, 'PUT': 2, 'P...",DC154,2,Put,PDC154P2,PDC154,1
2,2,6201146088473abe7f90507d,"{'_id': 6201146088473abe7f90507d, 'PUT': 1, 'P...",DC179,1,Put,PDC179P1,PDC179,2
3,3,6201146088473abe7f90507e,"{'_id': 6201146088473abe7f90507e, 'PUT': 2, 'P...",DC179,2,Put,PDC179P2,PDC179,3
4,4,6201146088473abe7f90507f,"{'_id': 6201146088473abe7f90507f, 'PUT': 3, 'P...",DC179,3,Put,PDC179P3,PDC179,4
5,5,6201146088473abe7f905080,"{'_id': 6201146088473abe7f905080, 'PUT': 4, 'P...",DC179,4,Put,PDC179P4,PDC179,5
6,6,6201146088473abe7f905081,"{'_id': 6201146088473abe7f905081, 'PUT': 5, 'P...",DC179,5,Put,PDC179P5,PDC179,6
7,7,6201146088473abe7f905082,"{'_id': 6201146088473abe7f905082, 'PUT': 6, 'P...",DC179,6,Put,PDC179P6,PDC179,7
8,8,6201146088473abe7f905083,"{'_id': 6201146088473abe7f905083, 'PUT': 7, 'P...",DC179,7,Put,PDC179P7,PDC179,8
9,9,6201146088473abe7f905084,"{'_id': 6201146088473abe7f905084, 'PUT': 8, 'P...",DC179,8,Put,PDC179P8,PDC179,9


In [41]:
df

,_id,brondata,projectcd,putnr,soort,key,key_project
0,6201144fbba1b3fdae51bef9,"{'_id': 6201144fbba1b3fdae51bef9, 'PUT': 1, 'P...",DC154,1,Put,PDC154P1,PDC154
1,6201144fbba1b3fdae51bef8,"{'_id': 6201144fbba1b3fdae51bef8, 'PUT': 2, 'P...",DC154,2,Put,PDC154P2,PDC154
2,6201146088473abe7f90507d,"{'_id': 6201146088473abe7f90507d, 'PUT': 1, 'P...",DC179,1,Put,PDC179P1,PDC179
3,6201146088473abe7f90507e,"{'_id': 6201146088473abe7f90507e, 'PUT': 2, 'P...",DC179,2,Put,PDC179P2,PDC179
4,6201146088473abe7f90507f,"{'_id': 6201146088473abe7f90507f, 'PUT': 3, 'P...",DC179,3,Put,PDC179P3,PDC179
5,6201146088473abe7f905080,"{'_id': 6201146088473abe7f905080, 'PUT': 4, 'P...",DC179,4,Put,PDC179P4,PDC179
6,6201146088473abe7f905081,"{'_id': 6201146088473abe7f905081, 'PUT': 5, 'P...",DC179,5,Put,PDC179P5,PDC179
7,6201146088473abe7f905082,"{'_id': 6201146088473abe7f905082, 'PUT': 6, 'P...",DC179,6,Put,PDC179P6,PDC179
8,6201146088473abe7f905083,"{'_id': 6201146088473abe7f905083, 'PUT': 7, 'P...",DC179,7,Put,PDC179P7,PDC179
9,6201146088473abe7f905084,"{'_id': 6201146088473abe7f905084, 'PUT': 8, 'P...",DC179,8,Put,PDC179P8,PDC179
